In [1]:
import pandas as pd

In [2]:
# Load surf spot data
surf_spots = pd.read_csv("Surf Spots.csv")
print(surf_spots.head())

            Name  Latitude  Longitude       Direction         Type  \
0     Arugam Bay  6.840739  81.836999           Right  Point-break   
1      Dalawella  5.999671  80.263658  Right and left   Reef-coral   
2    Devils rock  5.962863  80.379945            Left   Reef-rocky   
3  Dewata, Galle  6.032651  80.238465  Right and left  Point-break   
4  Elephant rock  6.810079  81.824244           Right  Point-break   

            Experience  
0          All surfers  
1  Experienced surfers  
2  Experienced surfers  
3          All surfers  
4          All surfers  


In [3]:
import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

In [4]:
# Function to create a rectangle buffer around each surf spot
def create_rectangle(lon, lat, buffer_size=0.01):
    """
    Create a rectangle around a point (lon, lat) with a given buffer size.
    Args:
    - lon: Longitude of the center point.
    - lat: Latitude of the center point.
    - buffer_size: Half the width/height of the rectangle in degrees.

    Returns:
    - ee.Geometry.Rectangle
    """
    return ee.Geometry.Rectangle([
        lon - buffer_size,  # Min longitude
        lat - buffer_size,  # Min latitude
        lon + buffer_size,  # Max longitude
        lat + buffer_size   # Max latitude
    ])

In [5]:
# Convert surf spots to GEE FeatureCollection with rectangle buffers
features = []
buffer_size = 0.01  # Adjust buffer size (~1 km)
for _, row in surf_spots.iterrows():
    rectangle = create_rectangle(row['Longitude'], row['Latitude'], buffer_size)
    feature = ee.Feature(rectangle, {'surf_spot_name': row['Name']})
    features.append(feature)

surf_spots_fc = ee.FeatureCollection(features)

In [6]:
# Load Sentinel-1 data
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(surf_spots_fc) \
    .filterDate("2023-01-01", "2023-12-31") \
    .filter(ee.Filter.eq("instrumentMode", "IW")) \
    .select(["VV", "VH"])

In [7]:
# Apply water mask
water_mask = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gt(0)
masked_sentinel1 = sentinel1.map(lambda img: img.updateMask(water_mask))

In [22]:
# Function to extract VV and VH data for each surf spot and date
def extract_sentinel_data_per_date(image):
    image_date = image.date().format("YYYY-MM-dd")  # Extract date from image
    
    def map_feature(feature):
        geom = feature.geometry()
        values = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geom,
            scale=50,
            maxPixels=1e9
        )
        return feature.set(values).set({"date": image_date})  # Add date to feature properties
    
    return surf_spots_fc.map(map_feature)

In [24]:
# Iterate over each image in the Sentinel-1 collection
surf_spots_with_data = masked_sentinel1.map(extract_sentinel_data_per_date).flatten()

In [26]:
# Export the data to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=surf_spots_with_data,
    description="SurfSpots_Sentinel1_Rectangle_WaterMask1",
    fileFormat="CSV"
)
task.start()

print("Export task started. Check your Google Drive for the output file.")

Export task started. Check your Google Drive for the output file.
